In [ ]:
!pip install datasets trl peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "1"

# full fine tune
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer

# we are trying to do text classification using SFT
dataset = load_dataset("stanfordnlp/imdb", split="test")
train_dataset = dataset.select(range(100))
test_dataset = dataset.select(range(100,110))

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
training_args = SFTConfig(
    dataset_text_field="text",
    packing=True,
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    output_dir="/imdb-model",
    eval_strategy="epoch",
    fp16=True
)
trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    args=training_args,
)
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:202: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,3.390800
2,No log,3.425132
3,No log,3.481913


TrainOutput(global_step=90, training_loss=2.8122745090060763, metrics={'train_runtime': 113.1367, 'train_samples_per_second': 0.795, 'train_steps_per_second': 0.795, 'total_flos': 167744813137920.0, 'train_loss': 2.8122745090060763, 'epoch': 3.0})

In [ ]:
# lora, qlora
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
from peft import get_peft_model
from peft import LoraConfig
import torch


lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_storage=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-350m",
    quantization_config=quantization_config
)
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

`low_cpu_mem_usage` was None, now default to True since model is quantized.


trainable params: 786,432 || all params: 331,982,848 || trainable%: 0.2369


In [ ]:
trainer = SFTTrainer(
    model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    args=training_args,
)
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,3.402172
2,No log,3.387527
3,No log,3.383702


TrainOutput(global_step=90, training_loss=3.6435831705729167, metrics={'train_runtime': 24.5689, 'train_samples_per_second': 3.663, 'train_steps_per_second': 3.663, 'total_flos': 168179678576640.0, 'train_loss': 3.6435831705729167, 'epoch': 3.0})